In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.chdir('..')

In [2]:
# os.environ.get("ZHIPUAI_API_KEY")

# 设计和使用

## 命令清单

命令清单：
```python
_INVLIAD_COMMANDS = [
    "quit",       # 退出
    "text",       # 某ID下的文字成果，默认ROOT
    "all",        # 所有任务
    "todos",      # 所有待办
    "todo",       # 某ID待办，默认当前ID
    "ok",         # 确认INIT任务，或某ID提纲或段落，然后自动进入下一待办任务
    "children",   # 查看某ID提纲
    "words",      # 查看后修改某ID字数
    "title",      # 查看后修改某ID标题
    "howto",      # 查看后修改某ID扩写指南
    "summarise",  # 查看后修改某ID段落摘要
    "reload",     # 重新加载模型
    "memory",     # 某ID对话记忆，默认当前ID
    "store",      # 某ID对话历史，默认当前ID
    "ask",        # 向AI提问
]
```

## 核心设计

**关键概念**
- [x] focus概念：`<START>`, `<0>`, `<END>`
- [x] 提示语变量中包含历史对话，默认支持20轮次
- [x] 每个节点有独立的对话历史
- [x] 内容ID：在同一个对象中全局自增，如果重写会产生新的内容ID（这是为了与删除环不冲突）
- [ ] 删除环：每个修改命令将修改或删除前内容放入环形临时队列，以便恢复
- [ ] 恢复操作：不做回退，而是继续累加删除环
- [ ] 粘贴：先删除到删除环，再从修改环最后一个粘贴到指定位置

**指令模式**
- [ ] 创作是一个状态机，每个状态有一组当下可执行的命令
- [ ] 完整输入指令：`<focus> command prompt`
- [x] auto: 默认为`none`，将一步步询问；如果设为`all`则表示全自动生成
- [x] run 循环：进入一个输入和生成的对话循环
- [x] run 的 task 参数：直接将第一轮输入通过 task 参数给出
- [x] run 的 max_step 参数：执行最大步数后就退出（设置为1时可当作函数使用）
- [x] all指令：所有所有任务列表
- [x] todos指令：查看所有待办任务列表
- [ ] todo指令：重新生成指定项的扩写提纲或详细内容，默认为修改当前位置
- [x] ok指令：确认当前生成的内容为已完成，默认转移到下一个未完成项

**Web服务模式**
- [ ] 新项目对话
- [ ] 开启与对话绑定的创作
- [ ] 支持每一步指令的状态返回
- [ ] 支持流返回

**模型支持**
- [x] 支持手动创建LLM对象
- [x] 支持自动识别 ZHIPUAI_API_KEY 和 OPENAI_API_KEY，并自动加载包和构造LLM对象
- [ ] 支持文心一言、讯飞星火、通义千问等其他模型
- [ ] 支持私有部署大模型

**提示工程**
- 提示语中的提纲信息：
  - [x] 模式1：全路径选标题、扩写指南和内容摘要？
  - [ ] 模式2：祖先节点选标题，兄弟节点选标题、扩写指南和内容摘要，最近节点选文字详情？
- [ ] 提示语外部信息
- [ ] 提示语公共信息：在START节点配置
- [ ] 支持本地知识库检索
- [ ] 支持数据库查询
- [ ] 支持智能体询问

**导出模板**
- [ ] 导出提示语模板：包括任务提示、局部提示语等
- [ ] 导出提纲：导出JSON文件，包括文档树的标题、字数、扩写指南、内容摘要等

**导入模板**
- [ ] 导入提示语模板：从现有提示语目录中导入
- [ ] 导入提纲：从符合格式要求的JSON文件中
- [ ] 提纲仿写：从文件内容中提取出精确的仿写提纲
- [ ] 提纲优化：从文字内容中提取出段落摘要，反向拆解提纲，再给出优化建议

**导出成果**
- [ ] 文字合并：如何对待提纲中的标题，作为段落合并还是作为章节？
- [ ] 支持局部导出
- [ ] 支持仅导出大纲
- 按格式输出
  - [ ] markdown
  - [ ] word
  - [x] txt

**修改能力：**
- [ ] 删除：将内容及其子项剪切到删除环
- [ ] 排序：先剪切，再插入
- [ ] 重新生成提纲列表：若要重新生成提纲列表，必须手工清楚所有子项，否则禁止使用todo指令或对其修改
- [ ] todo指令：仅针对子项尚未完成的对象（如刚刚完成的，或其他兄弟节点）
- [ ] 修改提纲子项属性：需要单独使用title、words、howto等指令修改

**其他设计：**
- [ ] 支持对日志输出着色

## Bug

Bug清单：
- [x] START 记忆确实存在，但无法通过 START@memory 正确提取
- [x] ask 指令无法在当前节点重新生成
- [x] todos清单增加一个标记为当前任务的状态：`[当前任务]`, `* <未完成>`, `<已完成>`
- [x] is_completed 无法修改


# WritingProject

## 每一步都询问：使用默认的智谱AI

In [2]:
from langchain_chinese import WritingTask

# 使用默认的智谱AI推理
wp = WritingTask(auto="none")

In [3]:
wp.run("致孩子的一封信, 800字", max_steps=1)

Parent run f0f0c3b8-08a1-45a7-9592-6a8ec6acf813 not found for run 3ab5be21-f61f-41a5-8b27-bcee78f31d07. Treating as a root run.


[START]
```json
{
    "总字数要求": 800,
    "标题名称": "致孩子的一封信",
    "扩写指南": "本文是一封父母给孩子写的信，信中应当包含深切的关爱、对孩子成长过程中的点滴回忆、以及对孩子未来的期望和祝福。建议在写作中提及具体的亲子互动经历、孩子的一些重要转折点，以及父母对孩子的理解和支持。信的语气应是温馨、鼓励和充满爱意的。"
}
```


In [4]:
wp.run(max_steps=100)

[START]



👤:  ok


Parent run d92762dc-fd01-4d63-bc99-88506c4b2e9b not found for run 6a02841f-1663-4c21-b042-8e5de0c2ccd2. Treating as a root run.


明白了，如果需要进一步的帮助，请随时告诉我。

Parent run 4c12c578-fae0-46bf-a48f-25b0104ccd91 not found for run 0af77d15-ceaa-4c9c-8ef0-c2100549f475. Treating as a root run.


...

Parent run 7438bc42-f9e5-4570-ae5e-fa996d1d7c07 not found for run 2792cf11-5439-45b2-a795-980d7134a555. Treating as a root run.


明白。期待您的下一次询问，我会尽力为您提供帮助。祝好！
推理错误: Invalid json output: 明白。期待您的下一次询问，我会尽力为您提供帮助。祝好！


Parent run 2bfa427a-0b1f-43a7-8858-8b2623c8eb0d not found for run 6a935f37-4629-4731-9a9b-6b9358aba863. Treating as a root run.


好的，期待再次为您服务。保重！
推理错误: Invalid json output: 好的，期待再次为您服务。保重！


Exception: AI返回结果无法正确解析，已经超过 5 次，可能需要调整提示语模板了！！

In [6]:
wp.run("memory")

[1]
[HumanMessage(content='请开始！'), AIMessageChunk(content='```json\n{\n    "详细内容": "亲爱的孩子，最近家里发生了很多温馨的事情。那天下雨，我们一起在客厅做烘焙，你小手忙个不停，那份专注让我们都忘了时间。当你自豪地展示你亲手做的饼干时，我们心中充满了喜悦。还有你学习上的进步，你的每一次小考成绩的提高，都让我们深感骄傲。我们注意到，你不仅个子长高了，还变得更加独立和自信。每晚读书时，你总是能提出深刻的问题，让我们看到了你的思考和成长。我们想让你知道，无论你走到哪里，我们的关爱都会像影子一样陪伴着你。",\n    "内容摘要": "描述了家庭烘焙、孩子学习进步等温馨时刻，展现了父母对孩子成长的关注和爱。涉及的人物为父母和孩子，地点为家中，主要情节包括家庭活动和孩子成长变化。"\n}\n```', response_metadata={'finish_reason': 'stop', 'model': 'glm-4', 'created': 1715483456, 'index': 0, 'usage': {'prompt_tokens': 445, 'completion_tokens': 178, 'total_tokens': 623}}, id='8635248524313626046')]



👤:  title


<1> title 



👤:  todo


<1> is_completed False
[1]


...


👤:  Memory


[HumanMessage(content='请开始！'), AIMessageChunk(content='```json\n{\n    "详细内容": "亲爱的孩子，最近家里发生了很多温馨的事情。那天下雨，我们一起在客厅做烘焙，你小手忙个不停，那份专注让我们都忘了时间。当你自豪地展示你亲手做的饼干时，我们心中充满了喜悦。还有你学习上的进步，你的每一次小考成绩的提高，都让我们深感骄傲。我们注意到，你不仅个子长高了，还变得更加独立和自信。每晚读书时，你总是能提出深刻的问题，让我们看到了你的思考和成长。我们想让你知道，无论你走到哪里，我们的关爱都会像影子一样陪伴着你。",\n    "内容摘要": "描述了家庭烘焙、孩子学习进步等温馨时刻，展现了父母对孩子成长的关注和爱。涉及的人物为父母和孩子，地点为家中，主要情节包括家庭活动和孩子成长变化。"\n}\n```', response_metadata={'finish_reason': 'stop', 'model': 'glm-4', 'created': 1715483456, 'index': 0, 'usage': {'prompt_tokens': 445, 'completion_tokens': 178, 'total_tokens': 623}}, id='8635248524313626046')]



👤:  howto


<1> howto 



👤:  quit


In [ ]:
wp.run()

In [ ]:
wp.run("ok", max_steps=1)

In [ ]:
wp.run()

In [ ]:
wp.get_memory("START")

In [ ]:
wp.get_memory(0)

In [ ]:
wp.print_all()

## 全自动生成：使用默认的智谱AI

In [2]:
from langchain_chinese import WritingTask

# 使用默认的智谱AI推理
# 你需要准备好智谱AI的APIKEY
wp = WritingTask(auto="all")
wp.run("我今天扒了二牛的裤子，然后他一整天都不理我，帮我写一封1000字道歉信，我希望和他继续做好朋友，每段不要少于500字哦")

[START]
```json
{
    "总字数要求": 1000,
    "标题名称": "致二牛的诚挚道歉信",
    "扩写指南": "这封信需要表达出深刻的歉意，解释为何会做出这样的行为，以及承诺未来不会再发生类似的事情。信中应当包含以下内容：对二牛的称呼；具体事件的描述；对二牛情绪的理解和歉意；对自己的反思；对友谊的重视；未来的承诺；结束语。建议使用温暖、真诚的语气，避免使用过于玩笑的言辞。"
}
```
-------------------- TODOs --------------------
* [0] 约1000字 | 《致二牛的诚挚道歉信》
```json
{
    "大纲列表": [
        {
            "总字数要求": 500,
            "标题名称": "事件的回顾与歉意表达",
            "扩写指南": "在此部分，详细描述事件经过，向二牛表达诚挚的歉意，并尝试理解他的情绪感受。"
        },
        {
            "总字数要求": 500,
            "标题名称": "自我反思与未来承诺",
            "扩写指南": "这部分着重于自我反思，说明自己的失误和认识到的问题，同时承诺未来改进，并强调对友谊的重视。"
        }
    ]
}
```
-------------------- TODOs --------------------
* [1] 约500字 | 《事件的回顾与歉意表达》
* <2> 约500字 | 《自我反思与未来承诺》
```json
{
  "详细内容": "亲爱的二牛，我想要首先回顾那件事情的经过，我知道这对你我来说都不是容易的回忆。那是一个风和日丽的下午，我们本应享受美好的时光，却因为我的一时冲动和考虑不周，发生了那样不愉快的事情。我没有考虑到你的感受，在众人面前让你陷入了尴尬的境地。我现在深刻意识到，我的行为是多么的不妥，我真的很抱歉。我试图想象当时的你是多么的难过和失望，你的心情我完全能够理解。在这里，我衷心地向你道歉，希望你能感受到我的诚意。",
  "内容摘要": "此段详细描述了事件经过，向二牛表达了诚挚的歉意，并尝试理解他的情绪感受。涉及的人物为二牛，背景设定为一

In [9]:
wp.print_text()

《给二牛的一封道歉信》
总字数要求约1000字；已完成。
扩写指南 >>> 本信需要包含以下几个部分：首先，对扒裤子事件表示深刻的歉意，并详细描述自己的后悔和自责；其次，解释当时行为的动机和冲动，表明并非有意伤害二牛；接着，表达对二牛感受的理解和尊重，以及对他的不理睬表示的痛苦；最后，诚恳地请求二牛的原谅，并提出希望未来能继续维持和加强友谊。每段至少500字，以体现道歉的诚恳和重视。

1 深刻的歉意与后悔
 二牛，我在此向你表达我最深刻的歉意。回想起那日在众人面前扒下你的裤子，我心中充满了悔恨和自责。那行为不仅让你在那一刻失去了尊严，更在我们的友谊上留下了难以抹去的裂痕。每一个夜晚，我都会被这个记忆所困扰，想象着你的尴尬和痛苦，我对自己当时的冲动感到羞愧不已。如果时光可以倒流，我绝不会让那样的事情发生，我愿意用一切来换取你的原谅。
2 解释动机与冲动
 二牛，我必须向你坦白，那天在众人面前扒你裤子的行为，完全是出于一次冲动。当时，我们一群朋友在玩闹，气氛热烈，我脑中突然闪过一个念头，想要引起大家的注意，让气氛更加活跃。我完全没有考虑到这样的行为对你可能造成的伤害，这是我的失策，也是我最大的失误。我想要强调的是，我绝无任何恶意，更不是有意让你难堪。在那个瞬间，我的动机仅仅是想要寻求一些即兴的欢笑，却忽略了你的感受，我深感愧疚。


In [3]:
wp.run()

[END]


BaseException: (None, ' |Error Reply:', {'详细内容': '回想起那天的经过，我深感自责。我的言行无疑给您带来了困扰和伤害，我真诚地为我的失误向您道歉。自我反思之后，我认识到了自己的冲动和考虑不周，这些问题不仅影响了我们的关系，也让我自己感到懊悔。我承诺，在未来的日子里，我将更加注重言行，不再让类似的事情发生。我们之间的友谊对我来说无比珍贵，我愿意用行动来证明这一点，并希望得到您的理解和原谅。', '内容摘要': '此段详细描述了写信者对过去行为的自我反思，承认了错误并承诺改进。强调了友谊的重要性，并表达了对二牛的理解和歉意。涉及的人物为二牛，背景设定为一次因写信者冲动行为导致的纠纷。'})

## 查看记忆和对话历史

In [ ]:
[k for k in wp.memory._shorterm_memory_store]

In [ ]:
for x in wp.memory.get_shorterm_memory(session_id="20240508.180514.00002.626").chat_memory.messages:
    print("-"*40)
    print(x.content)

In [ ]:
wp.print_text()

## 全自动生成：更换为GPT4

In [ ]:
from langchain_openai import ChatOpenAI

# 使用 OpenAI/ChatGPT 推理
wp = WritingTask(task_mode="auto")
wp.run(llm = ChatOpenAI(model_name="gpt-4-turbo"))

In [ ]:
wp.print_lines()

In [ ]:
print(wp.root_content.get_outlines())

# 测试 WritingTask
## ask_user

In [2]:
from langchain_chinese import WritingTask

# 使用默认的智谱AI推理
wp = WritingTask(auto="none")
wp.ask_user("ok")

('START', 0, 'ok', '')

In [3]:
wp.ask_user("title 一首诗")

('START', 0, 'title', '一首诗')

In [4]:
wp.ask_user("<1> ask 帮我按两个段落")

('1', '1', 'ask', '帮我按两个段落')

In [5]:
wp.ask_user("<Start> ask 帮我按两个段落")

('START', 0, 'ask', '帮我按两个段落')

In [6]:
wp.ask_user("ask 帮我按两个段落")

('START', 0, 'ask', '帮我按两个段落')

# 测试 TreeContent

In [2]:
from langchain_chinese import TreeContent

root = TreeContent()
a = root.add_item(title="ABC")
b = root.add_item(title="XYZ")
print(a.id, a.title)
print(b.id, b.title)
print(root.id)
print(a.add_item().id)
print(b.add_item().id)
print(b.add_item().root.id)


1 ABC
2 XYZ
0
3
4
0


/Users/xuehongwei/github/langchain_chinese/langchain_chinese/writing/task.py:139: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.auto == "all" and self.focus is not "END" and self.ai_reply_json != {}:


In [9]:
root.get_item_by_id("4").id

4

In [10]:
root.get_item_by_id("3").id

3

In [5]:
root.current_state

AttributeError: 'TreeContent' object has no attribute 'model'

In [6]:
from statemachine import StateMachine, State

class ContentState(StateMachine):
    """实现基于有限状态机的内容管理"""

    # 定义有限状态机的状态
    #
    # 扩写指南
    init = State("init", initial=True)
    # 完全重新生成：有扩写指南，祖先已完成扩写
    todo = State("todo")
    # 已完成扩写
    done = State("done")
    # 重新生成：已完成扩写，支持锁定部份内容后修改，也可直接确认
    mod = State("mod")
    
    # 定义有限状态机的状态转换
    #
    init_todo = init.to(todo)

    todo_done = todo.to(done)
    done_todo = done.to(todo)

    done_mod = done.to(mod)
    mod_done = mod.to(done)

    mod_todo = mod.to(todo)
    
    def invalid_commands(self):
        """
        根据状态返回可用的指令集
        """
        commands = _COMMON_COMMANDS
        if self.state == "init":
            commands = list(set(commands + _READ_COMMANDS))
        elif self.state == "todo":
            commands = list(set(commands + _READ_COMMANDS + _WRITE_COMMANDS + _AI_CHAT_COMMANDS))
        elif self.state == "mod":
            commands = list(set(commands + _READ_COMMANDS + _WRITE_COMMANDS + _AI_CHAT_COMMANDS))
        elif self.state == "done":
            commands = list(set(commands + _READ_COMMANDS + _WRITE_COMMANDS))
        else:
            raise BaseException("Unknow conent STATE:", self.state)
        return commands


In [7]:
s = ContentState()

In [15]:
s.send("init_todo")

In [16]:
s.current_state

State('todo', id='todo', value='todo', initial=False, final=False)

In [ ]:
s.send("todo_done")

In [20]:
s.current_state

State('done', id='done', value='done', initial=False, final=False)

In [21]:
s.done_todo()

In [22]:
s.current_state

State('todo', id='todo', value='todo', initial=False, final=False)